In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from preprocess import Preprocess
import tensorflow as tf
import rrn

/Users/larry/Py3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('rating_data.csv')
df['freq'] = df.groupby('uid')['uid'].transform('count')  # count frequncy by column's values
df = df[df['freq'] > 5]  # remove row which corresponding frequence < 5

preprocess = Preprocess(df)

usr_matrix, item_matrix, ground_truth, batch_user, batch_item = preprocess.gen_batch()

In [3]:
for season_idx in range(ground_truth.shape[0]):
    for usr_idx in range(ground_truth.shape[1]):
        for item_idx in range(ground_truth.shape[2]):
            usr = preprocess.user_map[batch_user[usr_idx]]
            item = batch_item[item_idx] - 1
            try:
                assert ground_truth[season_idx][usr_idx][item_idx] == usr_matrix[usr_idx][season_idx][item]
            except AssertionError:
                print ("season_idx, usr_idx, item_idx: %d, %d, %d" % (season_idx, usr_idx, item))
            try:
                assert ground_truth[season_idx][usr_idx][item_idx] == item_matrix[item_idx][season_idx][usr]
            except AssertionError:
                print ("season_idx, usr, item_idx: %d, %d, %d" % (season_idx, usr, item_idx))

In [4]:
def get_hparas(name):
    if name == 'USER':
        hparas = {
            'NAME': name,
            'EMBED_UNITS': 40,
            'BATCH_SIZE': 64,
            'LSTM_UNITS': 40,
            'LATENT_UNITS': 20,
            'ITEM_NUM': 102,
            'TIME_ELAPSE': 25,
            'STATIONARY_LENGTH': 20,
        }
    elif name == 'ITEM':
        hparas = {
            'NAME': name,
            'EMBED_UNITS': 40,
            'BATCH_SIZE': 64,
            'LATENT_UNITS': 20,
            'LSTM_UNITS': 40,
            'TIME_ELAPSE': 25,
            'USER_NUM': 842,
            'STATIONARY_LENGTH': 20
        }
    
    return hparas

In [5]:
user_hparas = get_hparas('USER')
item_hparas = get_hparas('ITEM')

tf.reset_default_graph()

model = rrn.RRN(user_hparas=user_hparas, item_hparas=item_hparas, lr=0.001)

In [6]:
model.user_vars


[<tf.Variable 'ENCODE/ENCODE_USER/dense/kernel:0' shape=(102, 40) dtype=float32_ref>,
 <tf.Variable 'ENCODE/ENCODE_USER/dense/bias:0' shape=(40,) dtype=float32_ref>,
 <tf.Variable 'LSTM/LSTM_USER/rnn/basic_lstm_cell/kernel:0' shape=(80, 160) dtype=float32_ref>,
 <tf.Variable 'LSTM/LSTM_USER/rnn/basic_lstm_cell/bias:0' shape=(160,) dtype=float32_ref>,
 <tf.Variable 'AFFINE/AFFINE_USER/dense/kernel:0' shape=(40, 20) dtype=float32_ref>,
 <tf.Variable 'AFFINE/AFFINE_USER/dense/bias:0' shape=(20,) dtype=float32_ref>]

In [7]:
model.item_vars

[<tf.Variable 'ENCODE/ENCODE_ITEM/dense/kernel:0' shape=(842, 40) dtype=float32_ref>,
 <tf.Variable 'ENCODE/ENCODE_ITEM/dense/bias:0' shape=(40,) dtype=float32_ref>,
 <tf.Variable 'LSTM/LSTM_ITEM/rnn/basic_lstm_cell/kernel:0' shape=(80, 160) dtype=float32_ref>,
 <tf.Variable 'LSTM/LSTM_ITEM/rnn/basic_lstm_cell/bias:0' shape=(160,) dtype=float32_ref>,
 <tf.Variable 'AFFINE/AFFINE_ITEM/dense/kernel:0' shape=(40, 20) dtype=float32_ref>,
 <tf.Variable 'AFFINE/AFFINE_ITEM/dense/bias:0' shape=(20,) dtype=float32_ref>]

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter("logs/", sess.graph)